##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feature Engineering using TFX Pipeline and Pandas Transform

***Transform input data the PandasTransform TFX-Addons component.***

Note: We recommend running this tutorial in a Colab notebook, with no setup required!  Just click "Run in Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx-addons/blob/master/examples/PandasTransform.example.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx-addons/tree/master/examples/PandasTransform.example.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
</table></div>

This notebook provides an example of using the TFX-Addons Pandas Transform component to perform feature engineering.  See below for more information on Pandas Transform.

## Set Up
We first need to install the TFX Python package and download
the dataset which we will use for our model.

### Upgrade Pip

To avoid upgrading Pip in a system when running locally,
check to make sure that we are running in Colab.
Local systems can of course be upgraded separately.

In [1]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 10.6 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


### Install TFX and TFX-Addons


In [2]:
!pip install -q tfx tfx-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Did you restart the runtime?

If you are using Google Colab, the first time that you run
the cell above, you must restart the runtime by clicking
above "RESTART RUNTIME" button or using "Runtime > Restart
runtime ..." menu. This is because of the way that Colab
loads packages.

Check the TensorFlow, TFX, TFX-Addons, and Pandas versions.

In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import tfx_addons
print('TFX-Addons version: {}'.format(tfx_addons.__version__))
import pandas as pd
print('Pandas version: {}'.format(pd.__version__))

TensorFlow version: 2.9.1
TFX version: 1.9.1
TFX-Addons version: 0.2.4
Pandas version: 1.3.5


More Imports

In [2]:
import os
import pprint

import absl
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

### Set up variables

There are some variables used to define a pipeline. You can customize these
variables as you want. By default all output from the pipeline will be
generated under the current directory.

In [3]:
PIPELINE_NAME = "pandas-transform"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Prepare example data
We will download the example dataset for use in our TFX pipeline. The dataset
we are using is
[Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

We will use the **raw** Palmer Penguins dataset.

There are four numeric features in this dataset:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

Because the TFX ExampleGen component reads inputs from a directory, we need
to create a directory and copy the dataset to it.

In [4]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_path = 'https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_path, _data_filepath)

('/tmp/tfx-datazpl889sz/data.csv', <http.client.HTTPMessage at 0x7efe54f3af10>)

Take a quick look at what the raw data looks like.

In [5]:
!head {_data_filepath}

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
Adelie,Torgersen,39.1,18.7,181,3750,MALE
Adelie,Torgersen,39.5,17.4,186,3800,FEMALE
Adelie,Torgersen,40.3,18,195,3250,FEMALE
Adelie,Torgersen,NA,NA,NA,NA,NA
Adelie,Torgersen,36.7,19.3,193,3450,FEMALE
Adelie,Torgersen,39.3,20.6,190,3650,MALE
Adelie,Torgersen,38.9,17.8,181,3625,FEMALE
Adelie,Torgersen,39.2,19.6,195,4675,MALE
Adelie,Torgersen,34.1,18.1,193,3475,NA


There are some entries with missing values which are represented as `NA`.
We will just delete those entries in this tutorial.

In [6]:
!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
Adelie,Torgersen,39.1,18.7,181,3750,MALE
Adelie,Torgersen,39.5,17.4,186,3800,FEMALE
Adelie,Torgersen,40.3,18,195,3250,FEMALE
Adelie,Torgersen,36.7,19.3,193,3450,FEMALE
Adelie,Torgersen,39.3,20.6,190,3650,MALE
Adelie,Torgersen,38.9,17.8,181,3625,FEMALE
Adelie,Torgersen,39.2,19.6,195,4675,MALE
Adelie,Torgersen,41.1,17.6,182,3200,FEMALE
Adelie,Torgersen,38.6,21.2,191,3800,MALE


### Create the InteractiveContext
Last, we create an InteractiveContext, which will allow us to run TFX components interactively in this notebook.

In [7]:
context = InteractiveContext()

### Ingest the dataset

In [8]:
example_gen = tfx.components.CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/tfx-datazpl889sz/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Calculate and display the dataset statistics

In [9]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)
context.show(statistics_gen.outputs['statistics'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2022-08-11T23_50_52.905159-v90yotir/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2022-08-11T23_50_52.905159-v90yotir/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


### Infer the dataset schema

In [10]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2022-08-11T23_50_52.905159-v90yotir/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'body_mass_g',INT,required,single,-
'culmen_depth_mm',FLOAT,required,single,-
'culmen_length_mm',FLOAT,required,single,-
'flipper_length_mm',INT,required,single,-
'island',STRING,required,single,'island'
'sex',STRING,required,single,'sex'
'species',STRING,required,single,'species'


,Values
Domain,
'island',"'Biscoe', 'Dream', 'Torgersen'"
'sex',"'FEMALE', 'MALE', '.'"
'species',"'Adelie', 'Chinstrap', 'Gentoo'"


## Use PandasTranform to Perform Feature Engineering

PandasTransform enables users to perform feature engineering in dataframes, using either Pandas, Modin, Numpy, Scikit, or another library which supports dataframes.

**Important Note:** Because processing is distributed by Beam across a cluster of compute nodes, each invocation of PandasTransform will recieve a subset of the data. That means that **operations which require full passes over the dataset are not supported.**

If you require full passes over the dataset, you are encouraged to use TensorFlow Transform instead.  However, since PandasTransform is designed to receive a dict of the summary statistics which are created by StatisticsGen (or other tooling) then you can often use those values to avoid making a full pass over the data.

**Important Note:** Unlike TensorFlow Transform, PandasTransform does not create a preprocessing TensorFlow graph to be prepended to your trained model.  That means that you **MUST** perform the equivalent preprocessing in your serving client or a similar location in your system.

In [11]:
module_file = 'my_module_file.py'

We will normalize the numeric features using Z-score.

In [12]:
%%writefile {module_file}

import numpy as np
from absl import logging

def preprocessing_fn(df, schema, statistics):
  def zscore(x, stats=None):
    if x.isnull()[0] or stats[x.name]['std_dev'] == 0.0:
      return x
    else:
      y = (x - stats[x.name]['mean']) / stats[x.name]['std_dev']
      return y.astype(x.dtype) # to maintain type consistency
    
  logging.info('  Running user code with {} rows in a {}'.format(len(df), type(df)))
  numeric_cols = df.select_dtypes(include=[np.number]).columns
  df = df[numeric_cols].apply(zscore, stats=statistics)
  return df

Writing my_module_file.py


In [ ]:
from tfx_addons.pandas_transform import PandasTransform

panda = PandasTransform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    statistics=statistics_gen.outputs['statistics'],
    module_file = os.path.abspath(module_file)
)
context.run(panda, enable_cache=False)

Examine the transformed dataset

In [ ]:
statistics_gen2 = tfx.components.StatisticsGen(
    examples=panda.outputs['transformed_examples'])
context.run(statistics_gen2, enable_cache=False)
context.show(statistics_gen2.outputs['statistics'])